In [1]:
import numpy as np
import itertools
import datetime

import tensorflow as tf
import sklearn as sk

from gensim.models import Word2Vec

import Functions as fn
import Iterator as it
from DS import DS
from Set import pool
from FFModel import FF_Model
from RNNModel import RNN_Model

c:\users\luka\anaconda3\envs\tensorflow13\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


# Data Loading

In [2]:
Dataset = pool()
Dataset.load_texts('raw_texts')
Dataset.load_labels('raw_labels')

Raw Text Load Complete
Raw Labels Load Complete


In [3]:
target_dict = fn.load_labels('final_meta/labels')

Label Load Complete


# Embedding Generation

In [4]:
train_set = pool(data=(Dataset.get_DS(stage='test', labelled='yes')).data[:-10])
validation_set = pool(data=(Dataset.get_DS(stage='test', labelled='yes')).data[-10:])
test_set = Dataset.get_DS(stage='train', labelled='yes')
set_1 = Dataset.get_DS(stage='train', labelled='no')
set_2 = Dataset.get_DS(stage='test', labelled='no')
set_1.append(set_2.data)
set_1.append(train_set.data)
emb_set = set_1
print(emb_set.size, train_set.size, validation_set.size, test_set.size)

4585 238 10 10


In [5]:
#emb_set.process_for_embedding()
#sentences = emb_set.get_sentences()
#fn.write_sentences(sentences, 'final_meta/sentences')
sentences = fn.load_sentences('final_meta/sentences')

#model = Word2Vec(sentences, min_count=1, size=100)
#model.save('final_meta/W2V')
model = Word2Vec.load('final_meta/W2V')

vocab = model.wv.vocab.keys()

Sentence Load Complete


# Layer and Index Loading

In [6]:
#word_indices, emb_layer = fn.get_index_and_emb_layer(model)
#fn.write_word_indices(word_indices, 's2s/word_indices')
#fn.write_emb_layer(emb_layer, 's2s/emb_layer')

word_indices = fn.load_word_indices('s2s/word_indices')
emb_layer = fn.load_emb_layer('s2s/emb_layer')

Word Indices Load Complete
Embedding Layer Load Complete


# ELS2S Testing

In [7]:
train_set.process_for_els2s_testing()
validation_set.process_for_els2s_testing()
test_set.process_for_els2s_testing()

In [41]:
entry = 1

print(test_set.data[0].inp_toks[entry])
print(len(test_set.data[0].inp_toks[entry]))
print(test_set.data[0].inp_words[entry])
print(len(test_set.data[0].inp_words[entry]))
print(test_set.data[0].inp_labels[entry])
print(len(test_set.data[0].inp_labels[entry]))
print(test_set.data[0].out_labels[entry])
print(len(test_set.data[0].out_labels[entry]))

['insulin', 'nph']
2
['inr', 'of', '<num>.<num>', ',', 'last', 'target', '<num>.<num>', ',', 'then', 'received', '<num>', 'mg', 'in', 'evening', 'x', '<num>', 'additionally', ',', 'percocet', '<num>-<num>', 'tablets', 'po', 'q', '<num>', 'prn', ',', 'colace', '<num>', 'mg', 'po', 'bid', ',', 'insulin', 'nph', '<num>', 'units', 'subcu', 'bid', ',', 'sliding', 'scale', 'insulin', 'subcu', 'q', '<num>', ',', 'isordil', '<num>', 'mg', 'tid', ',', 'zestril', '<num>', 'mg', 'q', 'd', ',', 'lopressor', '<num>', 'mg', 'bid', ',', 'axid', '<num>', 'mg', 'po', 'bid', 'she', 'will', 'follow', 'with', 'dr', 'noah', 'in']
74
[[0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [2], [2], [4], [4], [4], [4], [0], [0], [1], [2], [2], [3], [4], [4], [4], [0], [1], [2], [2], [3], [4], [0], [1], [1], [2], [2], [3], [4], [0], [2], [2], [1], [3], [4], [4], [0], [1], [2], [2], [4], [0], [1], [2], [2], [4], [4], [0], [1], [2], [2], [4], [0], [1], [2], [2], [3], [4], [0], [0], [0], [0], [0], [0], [0], [0], [0],

In [42]:
print(' '.join(test_set.data[0].inp_words[entry]))

inr of <num>.<num> , last target <num>.<num> , then received <num> mg in evening x <num> additionally , percocet <num>-<num> tablets po q <num> prn , colace <num> mg po bid , insulin nph <num> units subcu bid , sliding scale insulin subcu q <num> , isordil <num> mg tid , zestril <num> mg q d , lopressor <num> mg bid , axid <num> mg po bid she will follow with dr noah in


In [9]:
max_tok, max_inp = 0, 0
for setin in [train_set, validation_set, test_set]:
    for case in setin.data:
        for inp_words in case.inp_words:
            max_inp = max(max_inp, len(inp_words))
        for inp_tok in case.inp_toks:
            max_tok = max(max_tok, len(inp_tok))
print(max_tok, max_inp)

14 94


In [11]:
sets = {}
sets['train'] = train_set.get_els2s_sets(word_indices, max_tok, max_inp)
sets['validation'] = validation_set.get_els2s_sets(word_indices, max_tok, max_inp)
sets['test'] = test_set.get_els2s_sets(word_indices, max_tok, max_inp)

In [12]:
entry = 19

print(sets['test'][0][19])
print(sets['test'][1][19])
print(len(sets['test'][1][19]))
print(sets['test'][2][19])
print(len(sets['test'][2][19]))
print(sets['test'][3][19])
print(sets['test'][4][19])
print(sets['test'][5][19])
print(len(sets['test'][5][19]))

[26054, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[8424, 34144, 24390, 15338, 5944, 24390, 39594, 48602, 35529, 12093, 27193, 24390, 5702, 5944, 4324, 4560, 34798, 35529, 29647, 20302, 12646, 24390, 24379, 39594, 11567, 19877, 9609, 11228, 26054, 5702, 16969, 1848, 20418, 41595, 4560, 42508, 5702, 16969, 14636, 19084, 26517, 39692, 673, 12646, 10311, 15131, 35388, 15448, 47267, 45997, 48602, 19089, 4560, 2106, 35626, 42958, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
94
[[6], [6], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [1], [2], [2], [3], [4], [4], [0], [1], [2], [2], [3], [4], [0], [0], [1], [0], [0], [5], [5], [5], [5], [5], [5], [0], [0], [1], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0

In [15]:
import tensorflow as tf
import numpy as np
import warnings
import matplotlib.pyplot as plt
import sklearn as sk

from Iterator import ELS2SIterator


class ELS2S_Model:

    def __init__(self, 
                 decay=0,
                 batch=50,
                 enc_vocab_size=100, 
                 dec_vocab_size=9, 
                 enc_emb_size=100, 
                 dec_emb_size=100, 
                 state_size=100, 
                 dropout=1.0,
                 learn_rate=0.001,
                 max_gradient_norm=5,
                 enc_emb_layer=False):
        self.decay = decay
        self.batch = batch
        self.enc_vocab_size = enc_vocab_size
        self.dec_vocab_size = dec_vocab_size
        self.enc_emb_size = enc_emb_size
        self.dec_emb_size = dec_emb_size
        self.state_size = state_size
        self.dropout = dropout
        self.learn_rate = learn_rate
        self.max_gradient_norm=max_gradient_norm
        self.enc_emb_layer = enc_emb_layer
        self.graph = None
        self.sess = None

    def reset_graph(self):
        if self.sess:
            self.sess.close()
        tf.reset_default_graph()

    def build_graph(self):
        self.reset_graph()

        # Placeholders
        targs = tf.placeholder(dtype=tf.int32, shape=[self.batch, 14])  # [batch_size, num_steps]
        toks = tf.placeholder(dtype=tf.int32, shape=[self.batch, 94])
        labs = tf.placeholder(dtype=tf.float32, shape=[self.batch, 94, 1])
        inp_seqlen = tf.placeholder(tf.int32, shape=[self.batch])
        y = tf.placeholder(dtype=tf.int32, shape=[self.batch, 94])
        #target_weights = tf.placeholder(tf.float32, shape=[self.batch, 94])
        #keep_prob = tf.placeholder(tf.float32)
        
        # Embeddings
        if type(self.enc_emb_layer) != bool:
            enc_embeddings = tf.get_variable('enc_embedding_matrix', [self.enc_vocab_size, self.enc_emb_size], dtype=tf.float32)
        else:
            enc_embeddings = tf.get_variable('enc_embedding_matrix', initializer=tf.constant(self.enc_emb_layer))
        taremb = tf.nn.embedding_lookup(enc_embeddings, targs)
        tarbow = tf.reduce_sum(taremb, 1)
        tarbowlab = tf.concat([tarbow, tf.constant(1, dtype=tf.float32, shape=[self.batch, 1])], axis=-1)
        
        tokind = tf.nn.embedding_lookup(enc_embeddings, toks)
        rnn_inputs = tf.concat([tokind, labs], axis=-1)
        
        # Bidirectional
        forward_cell = tf.nn.rnn_cell.GRUCell(self.state_size)
        #fw_zero_state = forward_cell.zero_state(batch_size=self.batch, dtype=tf.float32)
        #fw_init_state = fw_zero_state.clone(cell_state=tarbowlab)
        
        backward_cell = tf.nn.rnn_cell.GRUCell(self.state_size)
        #bw_zero_state = backward_cell.zero_state(batch_size=self.batch, dtype=tf.float32)
        #bw_init_state = bw_zero_state.clone(cell_state=tarbowlab)

        bi_outputs, encoder_state = tf.nn.bidirectional_dynamic_rnn(cell_fw=forward_cell,
                                                                    cell_bw=backward_cell, 
                                                                    inputs=rnn_inputs,
                                                                    sequence_length=inp_seqlen,
                                                                    initial_state_fw=tarbowlab,
                                                                    initial_state_bw=tarbowlab,
                                                                    dtype=tf.float32)
        
        # Conbining the output hidden states of ceells
        rnn_outputs = tf.concat(bi_outputs, -1)
        
        with tf.variable_scope('softmax'):
            W = tf.Variable(tf.truncated_normal(shape=[2*self.state_size, 7], stddev=0.05))
            b = tf.Variable(tf.constant(0.1, shape=[7]))
        new_rnn_shape = tf.reshape(rnn_outputs, shape=[-1, 2*self.state_size])
        logits = tf.matmul(new_rnn_shape, W) + b
        output = tf.reshape(logits, [self.batch, 94, 7])  
        
        preds = tf.nn.softmax(output)       
        prediction = tf.argmax(preds, 2)
        
        loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=output, labels=y))
        train_step = tf.train.AdamOptimizer(self.learn_rate).minimize(loss)

        self.graph = {'targs': targs,
                      'toks': toks,
                      'labs': labs,
                      'inp_seqlen': inp_seqlen,
                      'y': y,
                      'prediction': prediction,
                      'loss': loss,
                      'ts': train_step
                      }

    def train(self, sets, epochs=10, report_percentage=1, show_progress=False, show_plot=False):
        # Start a tf session and run the optimisation algorithm
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())

        trainer = ELS2SIterator(*(sets['train'][:-1]))

        #train_truth = sets['train'][4]
        #validation_truth = sets['validation'][4]
        #test_truth = sets['validation'][4]

        #train_feed = {self.graph['enc_x']: sets['train'][0],
                      #self.graph['enc_labs']: sets['train'][1],
                      #self.graph['enc_seqlen']: sets['train'][2],
                      #self.graph['dec_x']: sets['train'][3],
                      #self.graph['dec_seqlen']: sets['train'][4],
                      #self.graph['y']: sets['train'][5],
                      #self.graph['target_weights']: sets['train'][6],
                      #self.graph['keep_prob']: 1.0}
        #validation_feed = {self.graph['enc_x']: sets['validation'][0],
                           #self.graph['enc_labs']: sets['validation'][1],
                           #self.graph['enc_seqlen']: sets['validation'][2],
                           #self.graph['dec_x']: sets['validation'][3],
                           #self.graph['dec_seqlen']: sets['validation'][4],
                           #self.graph['y']: sets['validation'][5],
                           #self.graph['target_weights']: sets['validation'][6],
                           #self.graph['keep_prob']: 1.0}
        #test_feed = {self.graph['enc_x']: sets['test'][0],
                     #self.graph['enc_labs']: sets['test'][1],
                     #self.graph['enc_seqlen']: sets['test'][2],
                     #self.graph['dec_x']: sets['test'][3],
                     #self.graph['dec_seqlen']: sets['test'][4],
                     #self.graph['y']: sets['test'][5],
                     #self.graph['target_weights']: sets['test'][6],
                     #self.graph['keep_prob']: 1.0}

        #train_f1_score = []
        #validation_f1_score = []

        mark = (epochs * (len(sets['train'][0]) // self.batch) * report_percentage) // 100
        check_point = []
        N = 0

        warnings.simplefilter("ignore")
        while trainer.epochs < epochs:
            trtar, trtok, trlab, trlen, trout = trainer.next_batch(self.batch)
            feed = {self.graph['targs']: trtar,
                    self.graph['toks']: trtok,
                    self.graph['labs']: trlab, 
                    self.graph['inp_seqlen']: trlen,
                    self.graph['y']: trout}
            if N % mark == 0:
                #prediction = self.sess.run(self.graph['prediction'], feed_dict=train_feed)
                #pred_cut = [pred[:end] for (pred, end) in zip(prediction, sets['train'][3])]
                #f1_sum = 0
                #for i in range(len(pred_cut)):
                #    f1_sum += sk.metrics.f1_score(train_truth[i], pred_cut[i], labels=[1, 2, 3, 4, 5, 6], average='micro')       
                #train_f1_score.append(f1_sum / len(pred_cut))
                #prediction = self.sess.run(self.graph['prediction'], feed_dict=validation_feed)
                #pred_cut = [pred[:end] for (pred, end) in zip(prediction, sets['validation'][3])]
                #f1_sum = 0
                #for i in range(len(pred_cut)):
                #    f1_sum += sk.metrics.f1_score(validation_truth[i], pred_cut[i], labels=[1, 2, 3, 4, 5, 6], average='micro')       
                #validation_f1_score.append(f1_sum / len(pred_cut))
                #check_point.append(N)
                loss = self.sess.run(self.graph['loss'], feed_dict=feed)
                print('Epoch: {}, Learn Rate: {:.7f}, Perplexity: {:.2f}'.format(trainer.epochs, self.learn_rate, loss))
                if show_progress: print("Progress: %d%%" % (N * report_percentage // mark), end="\r")
            self.sess.run(self.graph['ts'], feed_dict=feed)
            #self.learn_rate = self.learn_rate * 1/(1 + self.decay * trainer.epochs)
            N += 1
        warnings.simplefilter("default")

        #test_prediction = self.sess.run(self.graph['prediction'], feed_dict=test_feed)
        #pred_cut = [pred[:end] for (pred, end) in zip(prediction, sets['test'][3])]
        #f1_sum = 0
        #for i in range(len(pred_cut)):
        #    f1_sum += sk.metrics.f1_score(test_truth[i], pred_cut[i], labels=[1, 2, 3, 4, 5, 6], average='micro')       
        #test_f1_score = f1_sum / len(pred_cut)
        
        #if show_progress:
        #    print('FInal Values: Tr-F1: {:.4f}, Val-F1: {:.4f}'.format(train_f1_score[-1], validation_f1_score[-1]))
        #    print("Test F1-Score: {:.4f}\n".format(test_f1_score))
        
        #if show_plot:
        #    np_check_point = np.array(check_point)
        #    np_train_f1 = np.array(train_f1_score)
        #    np_val_f1 = np.array(validation_f1_score)

        #    plt.plot(np_check_point, np_train_f1, label="Train")
        #    plt.plot(np_check_point, np_val_f1, label="Validation")
        #    plt.plot(np_check_point, np.ones(len(np_check_point))*0.35, label="Baseline")
        #    plt.xlabel("Batches")
        #    plt.ylabel("F1-Score")
        #    plt.legend()
        #    plt.show()
        
        #return train_f1_score, validation_f1_score, test_f1_score

    def predict(self, data):
        feed = {self.graph['targs']: data[0],
                self.graph['toks']: data[1],
                self.graph['labs']: data[2], 
                self.graph['y']: data[4], 
                self.graph['inp_seqlen']: data[3]}
        return self.sess.run(self.graph['prediction'], feed_dict=feed)

    def close(self):
        self.sess.close()

In [17]:
ELS2S = ELS2S_Model(decay = 0.000025,
                batch=50,
                enc_vocab_size=len(word_indices), 
                dec_vocab_size=len(word_indices), 
                enc_emb_size=100, 
                dec_emb_size=100, 
                state_size=101, 
                dropout=1.0,
                learn_rate=0.001,
                max_gradient_norm=5,
                enc_emb_layer=emb_layer)
ELS2S.build_graph()
ELS2S.train(sets=sets, epochs=100, report_percentage=1, show_progress=True, show_plot=True)

Epoch: 0, Learn Rate: 0.0010000, Perplexity: 1.94
Epoch: 1, Learn Rate: 0.0010000, Perplexity: 0.93
Epoch: 2, Learn Rate: 0.0010000, Perplexity: 0.81
Epoch: 3, Learn Rate: 0.0010000, Perplexity: 0.67
Epoch: 4, Learn Rate: 0.0010000, Perplexity: 0.57
Epoch: 5, Learn Rate: 0.0010000, Perplexity: 0.54
Epoch: 6, Learn Rate: 0.0010000, Perplexity: 0.43
Epoch: 7, Learn Rate: 0.0010000, Perplexity: 0.36
Epoch: 8, Learn Rate: 0.0010000, Perplexity: 0.29
Epoch: 9, Learn Rate: 0.0010000, Perplexity: 0.27
Epoch: 10, Learn Rate: 0.0010000, Perplexity: 0.22
Epoch: 11, Learn Rate: 0.0010000, Perplexity: 0.18
Epoch: 12, Learn Rate: 0.0010000, Perplexity: 0.17
Epoch: 13, Learn Rate: 0.0010000, Perplexity: 0.15
Epoch: 14, Learn Rate: 0.0010000, Perplexity: 0.12
Epoch: 15, Learn Rate: 0.0010000, Perplexity: 0.12
Epoch: 16, Learn Rate: 0.0010000, Perplexity: 0.10
Epoch: 17, Learn Rate: 0.0010000, Perplexity: 0.08
Epoch: 18, Learn Rate: 0.0010000, Perplexity: 0.07
Epoch: 19, Learn Rate: 0.0010000, Perplex

In [27]:
tru_seqs = [label for entry in sets['test'][4] for label in entry]
#for i in range(len(sets['test'][5])):
#    tru_vocab.append(sets['test'][5][i][:sets['test'][4][i]])

In [26]:
res = []
for i in range(len(sets['test'][0])):
    dummy = [[], [], [], [], [], []]
    for j in range(50):
        for k in range(6):
            dummy[k].append(sets['test'][k][i])
    temp = ELS2S.predict(dummy)
    res.append(temp[0])

In [32]:
res_seqs = [label for entry in res for label in entry]

In [49]:
sk.metrics.f1_score(tru_seqs, res_seqs, labels=[1, 2, 3, 4, 5, 6], average='micro')

0.82042494859492809

In [44]:
tru_seq_m = [1 if label==1 else 0 for label in tru_seqs]
tru_seq_do = [1 if label==2 else 0 for label in tru_seqs]
tru_seq_mo = [1 if label==3 else 0 for label in tru_seqs]
tru_seq_f = [1 if label==4 else 0 for label in tru_seqs]
tru_seq_du = [1 if label==5 else 0 for label in tru_seqs]
tru_seq_r = [1 if label==6 else 0 for label in tru_seqs]

res_seq_m = [1 if label==1 else 0 for label in res_seqs]
res_seq_do = [1 if label==2 else 0 for label in res_seqs]
res_seq_mo = [1 if label==3 else 0 for label in res_seqs]
res_seq_f = [1 if label==4 else 0 for label in res_seqs]
res_seq_du = [1 if label==5 else 0 for label in res_seqs]
res_seq_r = [1 if label==6 else 0 for label in res_seqs]

In [45]:
print('Med F1-Score: {:.3f}'.format(sk.metrics.f1_score(tru_seq_m, res_seq_m, average='binary')))
print('Dos F1-Score: {:.3f}'.format(sk.metrics.f1_score(tru_seq_do, res_seq_do, average='binary')))
print('Mod F1-Score: {:.3f}'.format(sk.metrics.f1_score(tru_seq_mo, res_seq_mo, average='binary')))
print('Fre F1-Score: {:.3f}'.format(sk.metrics.f1_score(tru_seq_f, res_seq_f, average='binary')))
print('Dur F1-Score: {:.3f}'.format(sk.metrics.f1_score(tru_seq_du, res_seq_du, average='binary')))
print('Rea F1-Score: {:.3f}'.format(sk.metrics.f1_score(tru_seq_r, res_seq_r, average='binary')))

Med F1-Score: 0.892
Dos F1-Score: 0.783
Mod F1-Score: 0.840
Fre F1-Score: 0.789
Dur F1-Score: 0.764
Rea F1-Score: 0.720


In [54]:
bla = 0

for i in range(len(test_set.data[0].inp_words[bla])):
    print('{}\t{}\t{}'.format(test_set.data[0].inp_words[bla][i], sets['test'][4][bla][i], res[bla][i]))

units	0	0
subcu	0	0
q	0	0
pm	0	0
supplemented	0	0
by	0	0
sliding	0	0
scale	0	0
regular	0	0
insulin	0	0
scale	0	0
,	0	0
isordil	0	0
<num>	0	0
mg	0	0
tid	0	0
,	0	0
zestril	0	0
<num>	0	0
mg	0	0
q	0	0
d	0	0
,	0	0
lopressor	0	0
<num>	0	0
mg	0	0
bid	0	0
,	0	0
axid	0	0
<num>	0	0
mg	0	0
bid	0	0
,	0	0
ofloxacin	1	1
<num>	2	2
mg	2	2
po	3	3
q	4	4
<num>	4	4
,	0	0
ecotrin	0	0
<num>	0	0
mg	0	0
q	0	0
d	0	0
,	0	0
vancomycin	0	0
<num>	0	0
mg	0	0
q	0	0
<num>.	0	0
she	0	0
reports	0	0
an	0	0
allergy	0	0
to	0	0
codeine	0	0
and	0	0
iodine	0	0


In [32]:
res_vocab = res
res_words = []
for case in res_vocab:
    temp = []
    for index in case:
        for word in word_indices:
            if word_indices[word] == index:
                temp.append(word)
    res_words.append(temp)

In [34]:
for i in range(360):
    if len(res_words[i]) != len(res_vocab[i]):
        print('nooo')

In [35]:
for i in range(5):
    print(res_words[i])
    print(tru_words[i])

['ofloxacin', '<num>', 'mg', 'po', 'q', '<num>', '<eos>']
['ofloxacin', '<num>', 'mg', 'po', 'q', '<num>', '<eos>']
['insulin', 'nph', '<num>', 'units', 'subcu', 'bid', '<eos>']
['insulin', 'nph', '<num>', 'units', 'subcu', 'bid', '<eos>']
['colace', '<num>', 'mg', 'po', 'bid', '<eos>']
['colace', '<num>', 'mg', 'po', 'bid', '<eos>']
['percocet', '<num>-<num>', 'tablets', 'po', 'q', '<num>', 'prn', '<eos>']
['percocet', '<num>-<num>', 'tablets', 'po', 'q', '<num>', 'prn', '<eos>']
['coumadin', '<eos>']
['coumadin', '<num>', 'mg', 'in', 'evening', 'x', '<num>', '<eos>']


In [42]:
saver = tf.train.Saver()
saver.save(S2S.sess, "s2s/model/model.ckpt")

's2s/model/model.ckpt'

In [59]:
loaded = S2S_Model(decay = 0.00001,
                batch=50,
                enc_vocab_size=len(word_indices), 
                dec_vocab_size=len(word_indices), 
                enc_emb_size=100, 
                dec_emb_size=100, 
                state_size=128, 
                dropout=1.0,
                learn_rate=0.001,
                max_gradient_norm=5,
                enc_emb_layer=emb_layer)
loaded.build_graph()
loaded.sess = tf.Session()
loader = tf.train.Saver()
loader.restore(loaded.sess, "s2s/model/model.ckpt")

c:\users\luka\anaconda3\envs\tensorflow13\lib\site-packages\tensorflow\python\util\tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if d.decorator_argspec is not None), _inspect.getargspec(target))


INFO:tensorflow:Restoring parameters from s2s/model/model.ckpt
